Firstly we scrape historical player data from pro-football-reference.com

In [7]:
import timeit
import time
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import string
import re

import warnings
warnings.filterwarnings('ignore')

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}

In [34]:
def player_name(soupy_object):
    try:
        pos = soupy_object.find('h1').text
    except:
        pos = 'N/A'
    return pos

def player_position(soupy_object):
    try:
        pos = soupy_object.find('strong', text=re.compile('Position')).findParent().text[-5:-3]
    except:
        pos = 'N/A'
    return pos

def player_height(soupy_object):
    try:
        height_parent = soupy_object.find('span', text=re.compile('lb')).parent()
        height = height_parent[0].text
    except:
        height = 'N/A'
    return height

def player_weight(soupy_object):
    try:
        weight_parent = soupy_object.find('span', text=re.compile('lb')).parent()
        weight = weight_parent[1].text
    except:
        weight = 'N/A'
    return weight

def player_season(soupy_object):
    try:
        season = []
        seasons = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for szn in seasons:
            season.append(szn['id'][-4:])
    except:
        season = []
    return season

def player_age(soupy_object):
    try:
        age = []
        ages = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for ag in ages:
            a = ag.find('td', attrs = {'data-stat':'age'}).text
            age.append(a)
    except:
        age = []
    return age

def player_games_played(soupy_object):
    try:
        games_played = []
        gamess = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for games in gamess:
            game = games.find('td', attrs = {'data-stat':'g'}).text
            games_played.append(game)
    except:
        games_played = []
    return games_played

def player_games_started(soupy_object):
    try:
        games_started = []
        gamess = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for games in gamess:
            game = games.find('td', attrs = {'data-stat':'gs'}).text
            games_started.append(game)
    except:
        games_started = []
    return games_started

def player_rush_att(soupy_object):
    try:
        rush_att = []
        rush_atts = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for rush_at in rush_atts:
            at = rush_at.find('td', attrs = {'data-stat':'rush_att'}).text
            rush_att.append(at)
    except:
        rush_att = []
    return rush_att

def player_rush_yds(soupy_object):
    try:
        rush_yds = []
        rush_ydss = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for rush_yd in rush_ydss:
            yd = rush_yd.find('td', attrs = {'data-stat':'rush_yds'}).text
            rush_yds.append(yd)
    except:
        rush_yds = []
    return rush_yds

def player_rush_tds(soupy_object):
    try:
        rush_tds = []
        rush_tdss = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for rush_td in rush_tdss:
            td = rush_td.find('td', attrs = {'data-stat':'rush_td'}).text
            rush_tds.append(td)
    except:
        rush_tds = []
    return rush_tds

def player_fumbles(soupy_object):
    try:
        fumble = []
        fumbles = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for fumbl in fumbles:
            f = fumbl.find('td', attrs = {'data-stat':'fumbles'}).text
            fumble.append(f)
    except:
        fumble = []
    return fumble

def player_targets(soupy_object):
    try:
        target = []
        targets = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for targ in targets:
            t = targ.find('td', attrs = {'data-stat':'targets'}).text
            target.append(t)
    except:
        target = []
    return target

def player_recs(soupy_object):
    try:
        rec = []
        recs = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for rc in recs:
            r = rc.find('td', attrs = {'data-stat':'rec'}).text
            rec.append(r)
    except:
        rec = []
    return rec

def player_rec_yds(soupy_object):
    try:
        rec_yd = []
        rec_yds = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for yds in rec_yds:
            yd = yds.find('td', attrs = {'data-stat':'rec_yds'}).text
            rec_yd.append(yd)
    except:
        rec_yd = []
    return rec_yd

def player_rec_tds(soupy_object):
    try:
        rec_td = []
        rec_tds = soupy_object.find_all('tr', attrs = {'id':re.compile('rushing_and_receiving.')})
        for tds in rec_tds:
            td = tds.find('td', attrs = {'data-stat':'rec_td'}).text
            rec_td.append(td)
    except:
        rec_td = []
    return rec_td

def player_pass_cmp(soupy_object):
    try:
        pass_cmp = []
        pass_cmps = soupy_object.find_all('tr', attrs = {'id':re.compile('passing.')})
        for pass_c in pass_cmps:
            cmp = pass_c.find('td', attrs = {'data-stat':'pass_cmp'}).text
            pass_cmp.append(cmp)
    except:
        pass_cmp = []
    return pass_cmp

def player_pass_att(soupy_object):
    try:
        pass_att = []
        pass_atts = soupy_object.find_all('tr', attrs = {'id':re.compile('passing.')})
        for pass_a in pass_atts:
            att = pass_a.find('td', attrs = {'data-stat':'pass_att'}).text
            pass_att.append(att)
    except:
        pass_att = []
    return pass_att

def player_pass_yds(soupy_object):
    try:
        pass_yd = []
        pass_yds = soupy_object.find_all('tr', attrs = {'id':re.compile('passing.')})
        for yds in pass_yds:
            yd = yds.find('td', attrs = {'data-stat':'pass_yds'}).text
            pass_yd.append(yd)
    except:
        pass_yd = []
    return pass_yd

def player_pass_tds(soupy_object):
    try:
        pass_td = []
        pass_tds = soupy_object.find_all('tr', attrs = {'id':re.compile('passing.')})
        for tds in pass_tds:
            td = tds.find('td', attrs = {'data-stat':'pass_td'}).text
            pass_td.append(td)
    except:
        pass_td = []
    return pass_td

def player_int(soupy_object):
    try:
        int = []
        ints = soupy_object.find_all('tr', attrs = {'id':re.compile('passing.')})
        for its in ints:
            i = its.find('td', attrs = {'data-stat':'pass_int'}).text
            int.append(i)
    except:
        int = []
    return int

def player_sacked(soupy_object):
    try:
        sack = []
        sacks = soupy_object.find_all('tr', attrs = {'id':re.compile('passing.')})
        for sk in sacks:
            s = sk.find('td', attrs = {'data-stat':'pass_sacked'}).text
            sack.append(s)
    except:
        sack = []
    return sack

In [35]:
# to test scraping functionalities
# url = f'https://www.pro-football-reference.com/players/A/AberWa00.htm'
# req = get(url, headers = headers)
# soup_get = BeautifulSoup(req.content, 'html.parser')

In [36]:
# name = player_name(soup_get)
# position = player_position(soup_get)
# height = player_height(soup_get)
# weight = player_weight(soup_get)

# season = player_season(soup_get)
# age = player_age(soup_get)
# games_played = player_games_played(soup_get)
# games_started = player_games_started(soup_get)
# rush_att = player_rush_att(soup_get)
# rush_yds = player_rush_yds(soup_get)
# rush_tds = player_rush_tds(soup_get)
# fumbles = player_fumbles(soup_get)
# targets = player_targets(soup_get)
# recs = player_recs(soup_get)
# rec_yds = player_rec_yds(soup_get)
# rec_tds = player_rec_tds(soup_get)
# pass_cmp = player_pass_cmp(soup_get)
# pass_att = player_pass_att(soup_get)
# pass_yds = player_pass_yds(soup_get)
# pass_tds = player_pass_tds(soup_get)
# ints = player_int(soup_get)
# sacked = player_sacked(soup_get)

# for i in range(len(season)):
#     targets[i]
    # data = {'Name': name, 'Position': position, 'Height': height, 'Weight': weight, 'Season': season[i], 'Age': age[i], 'Games_Played': games_played[i],
    #         'Game_Started': games_started[i], 'Rush_Attempts': rush_att[i], 'Rush_Yards': rush_yds[i], 'Rush_TDs': rush_tds[i], 'Fumbles': fumbles[i],
    #         'Targets': targets[i], 'Receptions': recs[i], 'Rec_Yards': rec_yds[i], 'Rec_TDs': rec_tds[i], 'Pass_Completions': '0',
    #         'Pass_Attempts': '0', 'Pass_Yards': '0', 'Pass_TDs': '0', 'Interceptions': '0', 'Sacked': '0'}
    

In [38]:
data_list = []
# rate limit needed because football reference blocks users that send more than 20 requests per minute
rate_limit = 4
def get_data():
    for letter in string.ascii_uppercase:
        timestart = timeit.default_timer()
        url = f'https://www.pro-football-reference.com/players/{letter}'
        req = get(url, headers = headers)
        # print(req)
        soup = BeautifulSoup(req.content, 'html.parser')
        links = soup.find_all('a', href=re.compile(f'players/{letter}'))
        # print(links)
        relevant_links = []
        for link in links:
            info = link.findParent().text[-14:]
            if info[0] != '(':
                continue
            elif info[1:3] not in ['QB', 'RB', 'WR', 'TE']:
                continue
            # receiver targets were recorded 1992 onwards which is why I chose this year
            elif info[-9:-5] == info[-4:] or int(info[-9:-5]) < 1992:
                continue
            else:
                relevant_links.append(link)
        # print('----------')
        # print(relevant_links)
        for k, item in enumerate(relevant_links):
            timestart_per_player = timeit.default_timer()
            main_url = 'https://www.pro-football-reference.com'
            sub_url = item.get('href')
            print(sub_url)
            data_url = main_url + sub_url
            request = get(data_url, headers=headers)
            soup_get = BeautifulSoup(request.content, 'html.parser')

            name = player_name(soup_get)
            position = player_position(soup_get)
            height = player_height(soup_get)
            weight = player_weight(soup_get)

            season = player_season(soup_get)
            age = player_age(soup_get)
            games_played = player_games_played(soup_get)
            games_started = player_games_started(soup_get)
            rush_att = player_rush_att(soup_get)
            rush_yds = player_rush_yds(soup_get)
            rush_tds = player_rush_tds(soup_get)
            fumbles = player_fumbles(soup_get)
            targets = player_targets(soup_get)
            recs = player_recs(soup_get)
            rec_yds = player_rec_yds(soup_get)
            rec_tds = player_rec_tds(soup_get)
            pass_cmp = player_pass_cmp(soup_get)
            pass_att = player_pass_att(soup_get)
            pass_yds = player_pass_yds(soup_get)
            pass_tds = player_pass_tds(soup_get)
            ints = player_int(soup_get)
            sacked = player_sacked(soup_get)

            if position == 'QB':
                if len(pass_cmp) != len(season):
                    timestop_per_player = timeit.default_timer()
                    if ((timestop_per_player - timestart_per_player) < rate_limit):
                        time.sleep(rate_limit - timestop_per_player + timestart_per_player)
                    continue
                for i in range(len(season)):
                    data = {'Name': name, 'Position': position, 'Height': height, 'Weight': weight, 'Season': season[i], 'Age': age[i], 'Games_Played': games_played[i],
                            'Game_Started': games_started[i], 'Rush_Attempts': rush_att[i], 'Rush_Yards': rush_yds[i], 'Rush_TDs': rush_tds[i], 'Fumbles': fumbles[i],
                            'Targets': '0', 'Receptions': '0', 'Rec_Yards': '0', 'Rec_TDs': '0', 'Pass_Completions': pass_cmp[i],
                            'Pass_Attempts': pass_att[i], 'Pass_Yards': pass_yds[i], 'Pass_TDs': pass_tds[i], 'Interceptions': ints[i], 'Sacked': sacked[i]}
                    data_list.append(data)
            else:
                for i in range(len(season)):
                    data = {'Name': name, 'Position': position, 'Height': height, 'Weight': weight, 'Season': season[i], 'Age': age[i], 'Games_Played': games_played[i],
                            'Game_Started': games_started[i], 'Rush_Attempts': rush_att[i], 'Rush_Yards': rush_yds[i], 'Rush_TDs': rush_tds[i], 'Fumbles': fumbles[i],
                            'Targets': targets[i], 'Receptions': recs[i], 'Rec_Yards': rec_yds[i], 'Rec_TDs': rec_tds[i], 'Pass_Completions': '0',
                            'Pass_Attempts': '0', 'Pass_Yards': '0', 'Pass_TDs': '0', 'Interceptions': '0', 'Sacked': '0'}
                    data_list.append(data)
            timestop_per_player = timeit.default_timer()
            if ((timestop_per_player - timestart_per_player) < rate_limit):
                time.sleep(rate_limit - timestop_per_player + timestart_per_player)
        timestop = timeit.default_timer()
        print(f'You scraped player names starting with: {letter}')
        print('Time :', timestop - timestart)
        
    return data_list

nfl_df = pd.DataFrame(get_data())
nfl_df.head()

/players/A/AbbrJa00.htm
/players/A/AbduKa00.htm
/players/A/AbduRa00.htm
/players/A/AdamCh00.htm
/players/A/AdamJe01.htm
/players/A/AdamJo03.htm
/players/A/AdamKy00.htm
/players/A/AddaJo00.htm
/players/A/AikeKa00.htm
/players/A/AikeSa00.htm
/players/A/AjayJa00.htm
/players/A/AjirSe00.htm
/players/A/AlexDa21.htm
/players/A/AlexDa00.htm
/players/A/AlexDe00.htm
/players/A/AlexKe00.htm
/players/A/AlexSh00.htm
/players/A/AlexSt00.htm
/players/A/AlexRi00.htm
/players/A/AlfoBr00.htm
/players/A/AlfoMa00.htm
/players/A/AlixCh00.htm
/players/A/AlleAn01.htm
/players/A/AlleAr00.htm
/players/A/AlleDa00.htm
/players/A/AlleDw00.htm
/players/A/AlleJa00.htm
/players/A/AlleJa01.htm
/players/A/AlliAu00.htm
/players/A/AlliGe01.htm
/players/A/AllrJo00.htm
/players/A/AlstMi00.htm
/players/A/AmarJa00.htm
/players/A/AmenDa00.htm
/players/A/AndeC.00.htm
/players/A/AndeCo00.htm
/players/A/AndeDa00.htm
/players/A/AndeDa01.htm
/players/A/AndeDe00.htm
/players/A/AndeJa00.htm
/players/A/AndeJo00.htm
/players/A/AndeM

,Name,Position,Height,Weight,Season,Age,Games_Played,Game_Started,Rush_Attempts,Rush_Yards,...,Targets,Receptions,Rec_Yards,Rec_TDs,Pass_Completions,Pass_Attempts,Pass_Yards,Pass_TDs,Interceptions,Sacked
0,\nKarim Abdul-Jabbar\n,RB,5-10,194lb,1996,22,16,14,307,1116,...,46,23,139,0,0,0,0,0,0,0
1,\nKarim Abdul-Jabbar\n,RB,5-10,194lb,1997,23,16,14,283,892,...,49,29,261,1,0,0,0,0,0,0
2,\nKarim Abdul-Jabbar\n,RB,5-10,194lb,1998,24,15,15,270,960,...,31,21,102,0,0,0,0,0,0,0
3,\nKarim Abdul-Jabbar\n,RB,5-10,194lb,1999,25,13,9,143,445,...,22,17,84,1,0,0,0,0,0,0
4,\nKarim Abdul-Jabbar\n,RB,5-10,194lb,1999,,10,6,115,350,...,16,13,59,1,0,0,0,0,0,0


In [39]:
nfl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5426 entries, 0 to 5425
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              5426 non-null   object
 1   Position          5426 non-null   object
 2   Height            5426 non-null   object
 3   Weight            5426 non-null   object
 4   Season            5426 non-null   object
 5   Age               5426 non-null   object
 6   Games_Played      5426 non-null   object
 7   Game_Started      5426 non-null   object
 8   Rush_Attempts     5426 non-null   object
 9   Rush_Yards        5426 non-null   object
 10  Rush_TDs          5426 non-null   object
 11  Fumbles           5426 non-null   object
 12  Targets           5426 non-null   object
 13  Receptions        5426 non-null   object
 14  Rec_Yards         5426 non-null   object
 15  Rec_TDs           5426 non-null   object
 16  Pass_Completions  5426 non-null   object
 17  Pass_Attempts 

In [40]:
nfl_df.to_csv('Datasets/nfl_player_data.csv', index_label = False)